# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
coordinates = {
    "latitude": [-31.9523, -33.8688, -27.4705],
    "longitude": [115.8613, 151.2093, 153.0260],
    "City": ["Perth", "Sydney", "Brisbane"],
    "Population": [2099530, 4637436, 2495825]
}

coordinates_df = pd.DataFrame(coordinates)
map_plot_1 = coordinates_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "OSM"
)

# Display the map plot_1
map_plot_1

c:\Users\kgndi\anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,cacequi,-29.8836,-54.8250,52.952,94,100,9.09,BR,1697075091
1,1,port-aux-francais,-49.3500,70.2167,34.808,94,100,3.95,TF,1697075091
2,2,al jawf,29.5000,38.7500,67.568,51,0,3.87,SA,1697075091
3,3,ushuaia,-54.8000,-68.3000,40.658,75,75,12.52,AR,1697075092
4,4,vallenar,-28.5708,-70.7581,66.074,36,0,0.37,CL,1697075092


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
#%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    #scale = 0.01,
    color = "City"
)

# Display the map plot
map_plot_1



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
max_temp_cond1 = (city_data_df["Max Temp"]<=80)
min_temp_cond1 = (city_data_df["Max Temp"]>=65)
humidity_cond1 = (city_data_df["Humidity"]<=70)

filtered_df = city_data_df.loc[max_temp_cond1 & min_temp_cond1 & humidity_cond1]
# Drop any rows with null values
filtered_df = filtered_df.dropna()

# Display sample data
filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,al jawf,29.5000,38.7500,67.568,51,0,3.87,SA,1697075091
4,4,vallenar,-28.5708,-70.7581,66.074,36,0,0.37,CL,1697075092
23,23,upington,-28.4478,21.2561,73.778,38,0,3.09,ZA,1697075093
28,28,awjilah,29.1081,21.2869,68.630,67,0,2.38,LY,1697075093
32,32,busan,35.1028,129.0403,66.182,52,20,2.06,KR,1697075093
...,...,...,...,...,...,...,...,...,...,...
517,517,jindabyne,-36.4167,148.6333,69.332,37,5,7.89,AU,1697075127
518,518,ginosa,40.5801,16.7561,67.820,68,0,0.29,IT,1697075127
531,531,ha'il,27.5219,41.6907,77.018,20,0,3.09,SA,1697075128
534,534,al kharijah,25.4514,30.5464,71.096,40,0,3.74,EG,1697075128


In [8]:
filtered_df.columns

Index(['City_ID', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date'],
      dtype='object')

### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
#copy so we don't lose original filtered_df
hotel_df = filtered_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,al jawf,SA,29.5000,38.7500,51,
4,vallenar,CL,-28.5708,-70.7581,36,
23,upington,ZA,-28.4478,21.2561,38,
28,awjilah,LY,29.1081,21.2869,67,
32,busan,KR,35.1028,129.0403,52,
...,...,...,...,...,...,...
517,jindabyne,AU,-36.4167,148.6333,37,
518,ginosa,IT,40.5801,16.7561,68,
531,ha'il,SA,27.5219,41.6907,20,
534,al kharijah,EG,25.4514,30.5464,40,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
#https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:140.6833,41.8833,10000&bias=proximity:140.6833,41.88338&limit=20&apiKey=

radius = 10000
params = {
    'categories' : "accommodation.hotel",
    "limit": 20,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
al jawf - nearest hotel: No hotel found
vallenar - nearest hotel: Hotel del Marqués
upington - nearest hotel: Die Eiland Holiday Ressort
awjilah - nearest hotel: No hotel found
busan - nearest hotel: No hotel found
nova sintra - nearest hotel: Residência Ka Dencho
cooma - nearest hotel: Royal Hotel
avarua - nearest hotel: Paradise Inn
zhengjiatun - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
burao - nearest hotel: Liiban hotel burco ( ENG:Shibiin )
as sulayyil - nearest hotel: No hotel found
laixi - nearest hotel: 豪邦度假村
saint-philippe - nearest hotel: Le Baril
umarizal - nearest hotel: Pousada Flores da Terra
baoqing - nearest hotel: No hotel found
benedita - nearest hotel: Quinta da Granja
twentynine palms - nearest hotel: Sunnyvale Garden Suites Hotel - Joshua Tree National Park
dera bugti - nearest hotel: No hotel found
punch - nearest hotel: No hotel found
hasaki - nearest hotel: 大新旅館
magugu - nearest hotel: No hotel found
port douglas

,City,Country,Lat,Lng,Humidity,Hotel Name
2,al jawf,SA,29.5000,38.7500,51,No hotel found
4,vallenar,CL,-28.5708,-70.7581,36,Hotel del Marqués
23,upington,ZA,-28.4478,21.2561,38,Die Eiland Holiday Ressort
28,awjilah,LY,29.1081,21.2869,67,No hotel found
32,busan,KR,35.1028,129.0403,52,No hotel found
...,...,...,...,...,...,...
517,jindabyne,AU,-36.4167,148.6333,37,Lake Jindabyne Hotel
518,ginosa,IT,40.5801,16.7561,68,Il Borgo Ritrovato
531,ha'il,SA,27.5219,41.6907,20,فندق كلاسيك
534,al kharijah,EG,25.4514,30.5464,40,Kharga Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    #scale = 0.01,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map plot
map_plot_2


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)